In [5]:
import pandas as pd
import numpy as np
import sklearn
import statsmodels.formula.api as smf
import numpy as np
import pandas as pd

In [12]:
def whatItIs(x):
    if x<.4:
        return "Swimsuit"
    if x<.7:
        return "Snorkelling mask"
    return "Surfboard"

N=1000
np.random.seed(100)
category=np.random.uniform(size=N)
myfunc=np.vectorize(whatItIs)
product_class=myfunc(category)

## SIMULATE WITH CONDITIONAL POISSON DISTRIBUTION
lam_pos = 4*(product_class=="Swimsuit") + 8*(product_class=="Snorkelling mask") + 1*(product_class=="Surfboard")
quantity=np.random.poisson(lam=lam_pos, size=N) + 1

In [13]:
customer_id=np.int32(np.random.uniform(low=120, high=600, size=N))
df=pd.DataFrame({'quantity': quantity, 'Product': product_class, 'customer_id': customer_id})
df.head(15)

,quantity,Product,customer_id
0,7,Snorkelling mask,345
1,3,Swimsuit,200
2,4,Snorkelling mask,245
3,2,Surfboard,485
4,3,Swimsuit,249
5,5,Swimsuit,548
6,7,Snorkelling mask,433
7,1,Surfboard,525
8,5,Swimsuit,425
9,14,Snorkelling mask,444


In [14]:
import math
from functools import reduce

class ProductOrder(object):
    """Records the relevant statistics of one or more orders."""
  
    def __init__(self, qty, **kwargs):
        self.prod_order = 1
        self.prod_q = qty

    def combine(self, other):
        """ Combines two ProductOrder objects into a single one
        which summarizes their data. 
        Returns an object of ProductOrder"""
        self.prod_order += other.prod_order
        self.prod_q += other.prod_q
        return self
    
    def get_mean(self):
        return self.prod_q / self.prod_order


class ProductCatalog(object):

    def __init__(self):
        self.products = {}

    def record_order(self, name, qty):
        item = ProductOrder(qty=qty)
        
        if name not in self.products:
            self.products[name] = item
        else:
            self.products[name] = self.products[name].combine(item)

    def get_product_aoq(self, name):
        product = self.products[name]
        return product.get_mean()


    def get_multi_product_aoq(self, prods):

        for i,p in enumerate(prods):
            #print(p)
            if i == 0:
                product = self.products[p]
            else:
                other_prod = self.products[p]
                product = product.combine(other_prod)

        return product.get_mean()


#######################################################
################   T E S T S    #######################
#######################################################
pc = ProductCatalog()
pc.record_order('SWIMSUIT', 10)
pc.record_order('SNORKELLING MASK', 5)
pc.record_order('SNORKELLING MASK', 1)
pc.record_order('SNORKELLING MASK', 1)
pc.record_order('SNORKELLING MASK', 3)
pc.record_order('SURFBOARD', 1)
pc.record_order('SURFBOARD', 19)
assert pc.get_product_aoq('SWIMSUIT') == 10
assert pc.get_product_aoq('SNORKELLING MASK') == 2.5

assert pc.get_multi_product_aoq(['SURFBOARD', 'SNORKELLING MASK']) == 5.0

assert pc.get_product_aoq('SNORKELLING MASK') == 2.5
print(pc.get_product_aoq('SURFBOARD'))
#assert pc.get_product_aoq('PLUSH_CHAIR') == 10
print("All Done!")

5.0
All Done!


In [18]:
import numpy as np
## This generates data for use in the next cell
all_prods=['Single Lens Mask', 'Full Face Mask', 'Panoramic Mask', 'ProDive Premium Mask',
    'Seaview 180 Mask', 'Phantom Aquatics Mask', 'Cressi F1 Mask',  'Cressi Pano Mask'
               'Paddleboard', 'Longboard', 'Shortboard', 'Fish Board', 'Funboard', 'Tow-In Board', 'Bodyboard',
               'Soft-top Board', 'Hybrid Board', 'Gun Board']
assert len(all_prods) == len(set(all_prods))
np.random.seed(508) 

pc = ProductCatalog()
rands=np.random.binomial(10, .1, len(all_prods))
for p, aoq in zip(all_prods, rands):
    pc.record_order(p, max(aoq, 1))

In [19]:
similar=dict()
similar['Single Lens Mask'] = ['Full Face Mask', 'Panoramic Mask', 'ProDive Premium Mask']
similar['ProDive Premium Mask'] = ['Seaview 180 Mask', 'Phantom Aquatics Mask']
similar['Cressi Pano Mask'] = ['Cressi F1 Mask']

similar['Paddleboard'] = ['Longboard', 'Fish Board', 'Soft-top Board']
similar['Shortboard'] = ['Funboard', 'Bodyboard', 'Tow-In Board']
similar['Bodyboard'] = ['Shortboard', 'Gun Board']
similar['Longboard'] = ['Paddleboard', 'Fish Board', 'Hybrid Board', 'Soft-top Board']
similar['Hybrid Board'] = ['Gun Board']

In [23]:
def find_proxy(thing, N):
    list = []
    n = 0
    i = 0
    s = set()
    for e in similar[thing]:
        list.append((e, 0))
        s.add(e)
        n += 1
    while (n < N):
        for (e, j) in list:
            if (j == i and e in similar.keys()):
                for q in similar[e]:
                    if q not in s and q != thing:
                        list.append((q, i + 1))
                        s.add(q)
                        n += 1
        i += 1
        print(n)
    a = 0
    print(s)
    for e in s:
        a += pc.get_product_aoq(e)
    return a / N
expects= [( 'Single Lens Mask', 3, 4/3),
          ( 'Single Lens Mask', 5, 7/5)
         ]

for e in expects:
    assert find_proxy(e[0], e[1]) == e[2], f"{e} {find_proxy(e[0], e[1])}"

print("All Done!")

{'Panoramic Mask', 'Full Face Mask', 'ProDive Premium Mask'}
{'Seaview 180 Mask', 'Full Face Mask', 'Panoramic Mask', 'ProDive Premium Mask', 'Phantom Aquatics Mask'}
All Done!


In [25]:
find_proxy('Paddleboard', 3)

{'Fish Board', 'Soft-top Board', 'Longboard'}


1.3333333333333333

# Water Vehicle Rental

In [ ]:
import datetime

class WaterVehicleRental:
    
    def __init__(self,stock=[0, 0, 0]):
        """
        Our constructor class that instantiates bike rental shop.
        """
        self.stock = {}
        self.prices = {"Boat":25, "Canoe":15, "Paddleboard":10}
        types = ["Boat", "Canoe",  "Paddleboard"]
        for l in range(0, len(types)):
            self.stock[types[l]] = stock[l]

    def displaystock(self):
        """
        Displays the number of each type of the vehicles currently available for rent in the shop.
        """
        for b in self.stock:
            print("We have currently {} {}s available to rent.".format(self.stock[b], b))
        return self.stock

    def rent_water_vehicle(self, type, n):
        """
        Rents a bike on hourly basis to a customer.
        """
        if n <= 0:
            print("Number should be positive!")
            return None
        
        elif n > self.stock[type]:
            print("Sorry! We have currently {} {}(s) available to rent.".format(self.stock[type], type))
            return None
        
        else:
            now = datetime.datetime.now()                      
            print("You have rented a {} {}(s) today at {}".format(n,type, now))
            print("Have a fun water trip!")

            self.stock[type] -= n
            return now      
    
    def return_water_vehicle(self, return_request):
        """
        1. Accept a rented bike from a customer
        2. Replensihes the inventory
        3. Return a bill
        """
        rentalTimes = return_request[0]
        items = return_request[1]
        bill = 0
        
        for v in items:
            if not items[v]:
                continue
            self.stock[v] += items[v]
            now = datetime.datetime.now()
            rentalPeriod = now - rentalTimes[v]
            
            s = max(1, round(rentalPeriod.seconds / 3600)) * self.prices[v] * items[v]
            
               
            if (3 <= items[v] <= 5):
                print("You are eligible for Family rental promotion of 20% discount for {} price".format(v))
                s = s * 0.8
            bill += s
        
        if bill:
            print("Thanks for returning the vehicles. Hope you enjoyed our service!")
            print("That would be ${}".format(bill))
            return bill
        else:
            print("Are you sure you rented something with us?")
            return None





In [ ]:
class Customer:

    def __init__(self):
        """
        Our constructor method which instantiates various customer objects.
        """
        
        self.vehicles = {}
        self.rentalTime = {}
        for l in ["Boat", "Canoe",  "Paddleboard"]:
            self.vehicles[l] = 0
            self.rentalTime[l] = 0
        self.bill = 0

    
    def request_water_vehicle(self, type):
        """
        Takes a request from the customer for the number of bikes.
        """
                      
        n = input("How many items would you like to rent? ")
        try:
            n = int(n)
        except ValueError:
            print("That's not a positive integer!")
            return -1

        if n < 1:
            print("Invalid input. Number of bikes should be greater than zero!")
            return -1
        else:
            self.vehicles[type] += n
        return n
     
    def return_all(self):
        """
        Allows customers to return all the rented vehicles to the shop.
        """
        r = 0
        for q in self.vehicles.values():
            r += q
        if self.rentalTime and r:
            return [self.rentalTime, self.vehicles]  
        else:
            return [0,{}]

In [83]:
shop = WaterVehicleRental([10, 10, 10])
customer = Customer()

while True:
    print("""
    ====== Water Sport Rental =======
    1. Display available vehicles
    2. Request a boat on hourly basis $25
    3. Request a canoe on hourly basis $15
    4. Request a paddleboard on hourly basis $10
    5. Return all water vehicles
    6. Exit
    """)

    choice = input("Enter choice: ")

    try:
        choice = int(choice)
    except ValueError:
        print("That's not an int!")
        continue

    if choice == 1:
        shop.displaystock()

    elif choice >= 2 and choice <= 4:
        type = list(shop.stock.keys())[choice - 2]
        customer.rentalTime[type] = shop.rent_water_vehicle(type, customer.request_water_vehicle(type))

    elif choice == 5:
        customer.bill = shop.return_water_vehicle(customer.return_all())
        customer.rentalTime, customer.vehicles = {},{}        
    elif choice == 6:
        break
    else:
        print("Invalid input. Please enter number between 1-6 ")        
print("Thank you for using the boat rental system.")



    ====== Water Sport Rental =======
    1. Display available vehicles
    2. Request a boat on hourly basis $25
    3. Request a canoe on hourly basis $15
    4. Request a paddleboard on hourly basis $10
    5. Return all water vehicles
    6. Exit
    
Enter choice: 1
We have currently 10 Boats available to rent.
We have currently 10 Canoes available to rent.
We have currently 10 Paddleboards available to rent.

    ====== Water Sport Rental =======
    1. Display available vehicles
    2. Request a boat on hourly basis $25
    3. Request a canoe on hourly basis $15
    4. Request a paddleboard on hourly basis $10
    5. Return all water vehicles
    6. Exit
    
Enter choice: 3
How many items would you like to rent? 2
You have rented a 2 Canoe(s) today at 2023-03-18 19:29:44.732927
Have a fun water trip!

    ====== Water Sport Rental =======
    1. Display available vehicles
    2. Request a boat on hourly basis $25
    3. Request a canoe on hourly basis $15
    4. Request a padd

In [85]:
shop.stock.keys()

dict_keys(['Boat', 'Canoe', 'Paddleboard'])